In [2]:
import f90nml

In [50]:
file_path = '../namelist.wps'
inp = f90nml.read(file_path)

In [77]:
doms = 4
start_date = '2018-04-05_06:00:00'
end_date = '2018-04-08_06:00:00'
interval_seconds = 6*3600

In [81]:
shar = inp['share']

shar['start_date']=[start_date]*doms 
shar['end_date']=[end_date]*doms 
shar['interval_seconds']=interval_seconds 

geo = inp['geogrid']
geo['parent_grid_ratio'] = [1,4,3,3]
geo['i_parent_start'] = [1,28,80,70]
geo['j_parent_start'] = [1,18,65,122]

geo['e_we']= [118, 253, 274, 100]
geo['e_sn']= [86, 205, 214, 79]
geo['dx']= 38000
geo['dy']= 38000
geo['map_proj']= 'mercator'
geo['ref_lat']= -17
geo['ref_lon']= -66.3
geo['stand_lon']= geo['ref_lon']

ung = inp['ungrib']

ung['prefix'] = 'PRESSURE'
ung['prefix'] = 'SURFACE'

met = inp['metgrid']
met['fg_name'] = ['SURFACE','PRESSURE']


In [82]:
inp

Namelist([('share',
           Namelist([('wrf_core', 'ARW'),
                     ('max_dom', 4),
                     ('start_date',
                      ['2018-04-05_06:00:00',
                       '2018-04-05_06:00:00',
                       '2018-04-05_06:00:00',
                       '2018-04-05_06:00:00']),
                     ('end_date',
                      ['2018-04-08_06:00:00',
                       '2018-04-08_06:00:00',
                       '2018-04-08_06:00:00',
                       '2018-04-08_06:00:00']),
                     ('interval_seconds', 21600),
                     ('io_form_geogrid', 2)])),
          ('geogrid',
           Namelist([('parent_id', [1, 1, 2, 3]),
                     ('parent_grid_ratio', [1, 4, 3, 3]),
                     ('i_parent_start', [1, 28, 80, 70]),
                     ('j_parent_start', [1, 18, 65, 122]),
                     ('e_we', [118, 253, 274, 100]),
                     ('e_sn', [86, 205, 214, 79]),
          

In [83]:
f90nml.write(inp,file_path,force=True)